In [18]:
import os
import sagemaker
import numpy as np
import tensorflow as tf
from sagemaker.tensorflow import TensorFlow

In [2]:
ON_SAGEMAKER_NOTEBOOK = True

train_instance_type='ml.p2.xlarge'
deploy_instance_type='ml.p2.xlarge'

bucket = 'sagemaker-mumbai-data-set'
key = 'ena_data'

In [3]:
train_input_path = "s3://{}/{}/train/".format(bucket, key)
validation_input_path = "s3://{}/{}/validation/".format(bucket, key)

In [4]:
sagemaker_session = sagemaker.Session()
if ON_SAGEMAKER_NOTEBOOK:
    role = sagemaker.get_execution_role()
else:
    role = '[IAM ROLE]'

In [5]:
hyperparameters = {'epochs': 1, 'batch_size' : 64}

In [6]:
os.path.abspath('')

'/home/ec2-user/SageMaker/sagemaker_imageclassify/script_mode'

In [12]:
estimator = TensorFlow(base_job_name='ena-classify-inceptionv3-4',
                       entry_point=os.path.join(os.path.abspath(''),'entry.py'),
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
#                        output_path='s3://sagemaker-mumbai-data-set/ena_data',
                       hyperparameters=hyperparameters,
                       train_instance_count=1, train_instance_type=train_instance_type)

In [13]:
print("Training ...")
local_inputs = {'train' : train_input_path, 
                'validation' : validation_input_path}
estimator.fit(local_inputs)

Training ...
2020-01-22 00:39:08 Starting - Starting the training job...
2020-01-22 00:39:09 Starting - Launching requested ML instances...
2020-01-22 00:40:06 Starting - Preparing the instances for training......
2020-01-22 00:40:52 Downloading - Downloading input data......
2020-01-22 00:41:44 Training - Downloading the training image..2020-01-22 00:42:20,811 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-01-22 00:42:21,303 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "batch_size": 64,
        "model_dir": "s3://sagemaker-ap-south-1-591532082772/ena-classify-inceptionv3

#015 1/63 [..............................] - ETA: 2:48 - loss: 2.2000 - accuracy: 0.2031#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/63 [..............................] - ETA: 1:34 - loss: 5.2939 - accuracy: 0.2500#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/63 [>.............................] - ETA: 1:09 - loss: 7.1764 - accuracy: 0.2396#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

2020-01-22 00:43:09.877618: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model successfully saved at: /opt/ml/model
2020-01-22 00:43:22,157 sagemaker-containers INFO     Reporting training SUCCESS

2020-01-22 00:43:38 Uploading - Uploading generated training model
2020-01-22 00:43:38 Completed - Training job completed
Training seconds: 166
Billable seconds: 166


## Deploying

In [45]:
from sagemaker.tensorflow.serving import Model

In [53]:
!ls

entry.py  src  train.ipynb


In [54]:
model = Model(entry_point='inference.py',
#               dependencies=['requirements.txt'],
              source_dir='./src',
              model_data='s3://sagemaker-ap-south-1-591532082772/ena-classify-inceptionv3-4-2020-01-22-00-39-07-889/output/model.tar.gz',
              framework_version='2.0.0', role=role)
predictor = model.deploy(initial_instance_count=1, endpoint_name='quikr_eye_v1', instance_type='ml.m4.xlarge')

ClientError: An error occurred (ValidationException) when calling the DescribeEndpointConfig operation: 1 validation error detected: Value 'quikr_eye_v1' at 'endpointConfigName' failed to satisfy constraint: Member must satisfy regular expression pattern: ^[a-zA-Z0-9](-*[a-zA-Z0-9])*